<a href="https://colab.research.google.com/github/skmisht/CNN/blob/main/CNN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Melanoma detection - PyTorch 

In [1]:
import os
import csv 
import zipfile
from google.colab import files

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Torch modules
import torch
print("Current Torch version: ", torch.__version__)
# to install if there is no latest version of torch
# !pip install -q torch==1.0.0 
import torchvision
from torchvision.transforms import transforms

Current Torch version:  1.7.0+cu101


In [2]:
# test if the notebook is running on GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# accessing data files from shared directory 
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/isic-challenge-2017")

Mounted at /content/drive


In [4]:
# list of files in isic-challenge folder
print(os.listdir('/content/drive/My Drive/isic-challenge-2017'))

['ISIC-2017_Training_Part3_GroundTruth.csv', 'ISIC-2017_Validation_Part3_GroundTruth.csv', 'ISIC-2017_Test_v2_Part3_GroundTruth.csv', 'ISIC-2017_Test_v2_Data', 'ISIC-2017_Training_Data', 'ISIC-2017_Validation_Data']


In [5]:
# assigning the variable to the training and also for the csv file
training_images = '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data'
# first 10 images from training
training_images_names = os.listdir(training_images)
print(training_images_names[:10])
trainign_groundTruth_csv = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Part3_GroundTruth.csv')
print(trainign_groundTruth_csv.head(5))

print() # blank line
# assigning the variable to the testing and also for the csv file
testing_images = '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data'
testing_images_names = os.listdir(testing_images)
print(testing_images_names[:10])
test_groundTruth_csv = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Part3_GroundTruth.csv')
print(test_groundTruth_csv.head(5))


print() # blank line
# Meta csv files from training & testing
training_meta_file = '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC-2017_Training_Data_metadata.csv'
testing_meta_file = '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data_metadata.csv' 
# display training meta data 
train_meta = pd.read_csv(training_meta_file, delimiter=',')
train_meta.head()

['ISIC_0013474.jpg', 'ISIC_0013474_superpixels.png', 'ISIC_0013480.jpg', 'ISIC_0013480_superpixels.png', 'ISIC_0013490_superpixels.png', 'ISIC_0013492_superpixels.png', 'ISIC_0013486.jpg', 'ISIC_0013489.jpg', 'ISIC_0013486_superpixels.png', 'ISIC_0013487.jpg']
       image_id  melanoma  seborrheic_keratosis
0  ISIC_0000000       0.0                   0.0
1  ISIC_0000001       0.0                   0.0
2  ISIC_0000002       1.0                   0.0
3  ISIC_0000003       0.0                   0.0
4  ISIC_0000004       1.0                   0.0

['ISIC_0013414.jpg', 'ISIC_0013414_superpixels.png', 'ISIC_0013455_superpixels.png', 'ISIC_0013416.jpg', 'ISIC_0013455.jpg', 'ISIC_0013416_superpixels.png', 'ISIC_0013457_superpixels.png', 'ISIC_0013457.jpg', 'ISIC_0013459.jpg', 'ISIC_0013465_superpixels.png']
       image_id  melanoma  seborrheic_keratosis
0  ISIC_0012086       0.0                   1.0
1  ISIC_0012092       0.0                   0.0
2  ISIC_0012095       0.0                   0

,image_id,age_approximate,sex
0,ISIC_0000000,55,female
1,ISIC_0000001,30,female
2,ISIC_0000002,60,female
3,ISIC_0000003,30,male
4,ISIC_0000004,80,male


In [6]:
# total images in training and testing directory
print("Total images in Training: ", len(training_images_names))
print("Total images in Testing: ", len(testing_images_names))

Total images in Training:  4001
Total images in Testing:  1201


# Joining Image to the Ground truth data